In [1]:
import pandas as pd
cdf = pd.read_parquet("abs_type_approvals.parquet")
cdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80765 entries, 0 to 80764
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   certificateNumber   80765 non-null  object 
 1   issueDate           79652 non-null  object 
 2   expirydate          79807 non-null  object 
 3   companyname         74034 non-null  object 
 4   customer_id         80765 non-null  object 
 5   address1            32130 non-null  object 
 6   country             74034 non-null  object 
 7   product             80765 non-null  object 
 8   description         80750 non-null  object 
 9   model               80765 non-null  object 
 10  modelcertId         80765 non-null  int32  
 11  absRules            80765 non-null  object 
 12  comments            80713 non-null  object 
 13  termsOfValidity     78394 non-null  object 
 14  serviceRestriction  80756 non-null  object 
 15  ratings             80727 non-null  object 
 16  cate

In [2]:
cdf['category'].unique()

array([None, 'Type Approved', 'EC Type Examined', 'Design Assessed',
       'Wheelmarked'], dtype=object)

In [20]:
cdf.columns

Index(['certificateNumber', 'issueDate', 'expirydate', 'companyname',
       'customer_id', 'address1', 'country', 'product', 'description', 'model',
       'modelcertId', 'absRules', 'comments', 'termsOfValidity',
       'serviceRestriction', 'ratings', 'category', 'eumed', 'other',
       'gov_authority', 'industry', 'nationals', 'state', 'business_phone',
       'zip', 'email', 'city', 'tier', 'Endorsement', 'intendedservice',
       'Auditnumber', 'typeofaudit'],
      dtype='object')

In [3]:
import base64, hashlib
def hash(x):
  return "id-" + base64.urlsafe_b64encode(hashlib.sha1(x).digest()).decode("utf-8") 

In [4]:
import json
import requests
from pprint import pprint

In [5]:
endpoint = 'https://gptkb-r5e7unxol46om.search.windows.net/'
api_version = '?api-version=2019-05-06'
headers = {'Content-Type': 'application/json',
        'api-key': 'kxgH5lLlChcWrsiGObjpjIrvwednYcUOaFgiZ12NXYAzSeBrN9g8' }

In [15]:
with open('typeapprovalindex.json') as json_file:
    index_schema = json.load(json_file)

In [13]:
#index_name="typeapprovalindex"
index_name="typeapprovalindex2"

In [8]:
#Thisis for deleting index
url = endpoint + "indexes/" + index_name + api_version
response  = requests.delete(url, headers=headers)
print(response)

<Response [204]>


In [17]:
url = endpoint + "indexes" + api_version
response  = requests.post(url, headers=headers, json=index_schema)
index = response.json()
pprint(index)

{'@odata.context': 'https://gptkb-r5e7unxol46om.search.windows.net/$metadata#indexes/$entity',
 '@odata.etag': '"0x8DBA3693EA489CF"',
 'analyzers': [],
 'charFilters': [],
 'corsOptions': {'allowedOrigins': ['*'], 'maxAgeInSeconds': 300},
 'defaultScoringProfile': None,
 'encryptionKey': None,
 'fields': [{'analyzer': None,
             'facetable': False,
             'filterable': False,
             'indexAnalyzer': None,
             'key': True,
             'name': 'id',
             'retrievable': True,
             'searchAnalyzer': None,
             'searchable': False,
             'sortable': False,
             'synonymMaps': [],
             'type': 'Edm.String'},
            {'analyzer': None,
             'facetable': False,
             'filterable': False,
             'indexAnalyzer': None,
             'key': False,
             'name': 'content',
             'retrievable': True,
             'searchAnalyzer': None,
             'searchable': True,
             'so

In [18]:
url = endpoint + "indexes" + api_version + "&$select=name"
response  = requests.get(url, headers=headers)
index_list = response.json()
pprint(index_list)

{'@odata.context': 'https://gptkb-r5e7unxol46om.search.windows.net/$metadata#indexes(name)',
 'value': [{'name': 'gptkbindex'},
           {'name': 'typeapprovalindex'},
           {'name': 'typeapprovalindex2'}]}


In [19]:
def ingest_docs(doc_df):
    doc_df = doc_df.fillna('')
    documents = {"value": doc_df.to_dict(orient="records")}
    url = endpoint + "indexes/" + index_name +"/docs/index" + api_version
    response  = requests.post(url, headers=headers, json=documents)
    response_df = pd.DataFrame.from_dict(response.json())
    return response_df

In [ ]:
cdf.head()

In [16]:
# id = modelCertId
# content = product + description + absRules + ratings
# category = category
# sourcepage = companyname + address1 + country
# sourcefile = certificateNumber
# issuedate = issueDate
# expirydate = expirydate

In [21]:
search_df = cdf[['modelcertId', 'product', 'description', 'absRules', 'ratings', 'category', 'companyname', 'address1', 'country', 'certificateNumber', 'issueDate', 'expirydate','eumed','other','gov_authority','industry','nationals']]
content_cols = ['product', 'description', 'absRules', 'ratings','eumed','other','gov_authority','industry','nationals']
company_cols = ['companyname', 'address1', 'country']

search_df['all'] = cdf.apply(lambda row: ' || '.join(row.values.astype(str)), axis=1)

search_df['content'] = search_df[content_cols].apply(lambda row: ' || '.join(row.values.astype(str)), axis=1)
search_df['company'] = search_df[company_cols].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)
search_df = search_df.drop(content_cols, axis=1)
search_df = search_df.drop(company_cols, axis=1)
search_df = search_df.rename(columns={'modelcertId': 'id', 'company': 'sourcepage', 'certificateNumber': 'sourcefile', 'issueDate': 'issuedate'})
search_df = search_df.dropna(subset=['issuedate', 'expirydate'])
search_df['issuedate'] = search_df['issuedate'].map(lambda x: x.strftime('%Y-%m-%dT%H:%M:%SZ'))
search_df['expirydate'] = search_df['expirydate'].map(lambda x: x.strftime('%Y-%m-%dT%H:%M:%SZ'))
search_df['id'] = search_df['id'].astype(str)
search_df.head()

/tmp/ipykernel_20117/2262666319.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  search_df['all'] = cdf.apply(lambda row: ' || '.join(row.values.astype(str)), axis=1)
/tmp/ipykernel_20117/2262666319.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  search_df['content'] = search_df[content_cols].apply(lambda row: ' || '.join(row.values.astype(str)), axis=1)
/tmp/ipykernel_20117/2262666319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,id,category,sourcefile,issuedate,expirydate,all,content,sourcepage
0,228663,None,08-SQ344138-PDA,2008-05-22T00:00:00Z,2013-05-21T00:00:00Z,08-SQ344138-PDA || 2008-05-22 || 2013-05-21 |...,Valve || See attachment || ABS SVR 2008 1-1-4/...,"BOTELI VALVE GROUP CO., LTD., None, China"
1,953,None,00-QA0505-LTD,2000-05-05T00:00:00Z,2004-08-01T00:00:00Z,00-QA0505-LTD || 2000-05-05 || 2004-08-01 || M...,"Mounting Devices for PCs and Monitors || 22"" F...","MARINER SYSTEMS UK LTD, None, United Kingdom"
2,337273,None,00-HS156975-3-DUP3,2010-07-19T00:00:00Z,2010-10-18T00:00:00Z,00-HS156975-3-DUP3 || 2010-07-19 || 2010-10-18...,Non-Metallic Flexible Hoses with End Fittings ...,GATES FLUID POWER TECHNOLOGIES (CHANGZHOU) CO....
3,372044,None,00-HS157686-3-PDA,2011-02-03T00:00:00Z,2011-05-22T00:00:00Z,00-HS157686-3-PDA || 2011-02-03 || 2011-05-22 ...,"Fire Suppression System, Inergen || Clean Agen...","TYCO FIRE PRODUCTS, LP, None, United States"
4,101472,None,00-HS175889A/1-PDA/DUP,2005-07-26T00:00:00Z,2006-09-25T00:00:00Z,00-HS175889A/1-PDA/DUP || 2005-07-26 || 2006-0...,"Valve, Cast Steel Swing Check || Sizes: as per...","HJ VALVE CO., LTD, 413, JUKGOK-RIJINYEONG-EUP,..."


In [22]:
search_df["@search.action"] = "upload"

In [23]:
search_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79652 entries, 0 to 80764
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              79652 non-null  object
 1   category        36592 non-null  object
 2   sourcefile      79652 non-null  object
 3   issuedate       79652 non-null  object
 4   expirydate      79652 non-null  object
 5   all             79652 non-null  object
 6   content         79652 non-null  object
 7   sourcepage      79652 non-null  object
 8   @search.action  79652 non-null  object
dtypes: object(9)
memory usage: 6.1+ MB


In [24]:
import numpy as np
partitions = 1000
data_splits = np.array_split(search_df, partitions)
for data_split in data_splits: 
    doc_df = data_split.fillna('')
    documents = {"value": doc_df.to_dict(orient="records")}
    url = endpoint + "indexes/" + index_name +"/docs/index" + api_version
    response  = requests.post(url, headers=headers, json=documents)
    #print(response.json())
    #break
    

In [25]:
search_df.to_parquet("typeapprovaldata1.parquet")